In [1]:
import pandas as pd , numpy as np
from tensorflow import keras

In [2]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

# Load the dataset
df = pd.read_csv(r"C:\Users\nadip\Downloads\archive (1)\Indian_housing_Delhi_data.csv")
df.head()

,house_type,house_size,location,city,latitude,longitude,price,currency,numBathrooms,numBalconies,isNegotiable,priceSqFt,verificationDate,description,SecurityDeposit,Status
0,1 RK Studio Apartment,400 sq ft,Kalkaji,Delhi,28.545561,77.254349,22000,INR,1.0,NaN,NaN,NaN,Posted a day ago,"Fully furnished, loaded with amenities & gadge...",No Deposit,Furnished
1,1 RK Studio Apartment,400 sq ft,Mansarover Garden,Delhi,28.643259,77.132828,20000,INR,1.0,NaN,NaN,NaN,Posted 9 days ago,Here is an excellent 1 BHK Independent Floor a...,No Deposit,Furnished
2,2 BHK Independent Floor,500 sq ft,Uttam Nagar,Delhi,28.618677,77.053352,8500,INR,1.0,NaN,NaN,NaN,Posted 12 days ago,"Zero Brokerage.\n\n2 Room set, Govt bijali Met...",No Deposit,Semi-Furnished
3,3 BHK Independent House,"1,020 sq ft",Model Town,Delhi,28.712898,77.180000,48000,INR,3.0,NaN,NaN,NaN,Posted a year ago,Itâs a 3 bhk independent house situated in M...,No Deposit,Furnished
4,2 BHK Apartment,810 sq ft,Sector 13 Rohini,Delhi,28.723539,77.131424,20000,INR,2.0,NaN,NaN,NaN,Posted a year ago,Well designed 2 bhk multistorey apartment is a...,No Deposit,Unfurnished


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   house_type        5000 non-null   object 
 1   house_size        5000 non-null   object 
 2   location          5000 non-null   object 
 3   city              5000 non-null   object 
 4   latitude          5000 non-null   float64
 5   longitude         5000 non-null   float64
 6   price             5000 non-null   int64  
 7   currency          5000 non-null   object 
 8   numBathrooms      4975 non-null   float64
 9   numBalconies      2737 non-null   float64
 10  isNegotiable      179 non-null    object 
 11  priceSqFt         0 non-null      float64
 12  verificationDate  5000 non-null   object 
 13  description       4715 non-null   object 
 14  SecurityDeposit   5000 non-null   object 
 15  Status            5000 non-null   object 
dtypes: float64(5), int64(1), object(10)
memory

In [4]:
df['house_size'] = df['house_size'].str.replace('sq ft', '').str.replace(',', '').astype(int)
df['house_size']

0        400
1        400
2        500
3       1020
4        810
        ... 
4995    5896
4996    6521
4997    1855
4998    2856
4999    2856
Name: house_size, Length: 5000, dtype: int32

In [5]:
df['SecurityDeposit'] = (df['SecurityDeposit'].replace('No Deposit', '0').str.strip().str.replace(',', '').astype(float))
print(df['SecurityDeposit'].isnull().sum())  
print(df['SecurityDeposit'].value_counts())

0
SecurityDeposit
0.0          2224
1010202.0     221
1810202.0      80
1020202.0      70
1010220.0      69
             ... 
1854425.0       1
1052122.0       1
4501105.0       1
4011012.0       1
5401015.0       1
Name: count, Length: 646, dtype: int64


In [6]:
df['numBathrooms'] = df['numBathrooms'].fillna(df['numBathrooms'].mode()[0])
df['numBathrooms'] = df['numBathrooms'].astype(int)

In [7]:
df.head()

,house_type,house_size,location,city,latitude,longitude,price,currency,numBathrooms,numBalconies,isNegotiable,priceSqFt,verificationDate,description,SecurityDeposit,Status
0,1 RK Studio Apartment,400,Kalkaji,Delhi,28.545561,77.254349,22000,INR,1,NaN,NaN,NaN,Posted a day ago,"Fully furnished, loaded with amenities & gadge...",0.0,Furnished
1,1 RK Studio Apartment,400,Mansarover Garden,Delhi,28.643259,77.132828,20000,INR,1,NaN,NaN,NaN,Posted 9 days ago,Here is an excellent 1 BHK Independent Floor a...,0.0,Furnished
2,2 BHK Independent Floor,500,Uttam Nagar,Delhi,28.618677,77.053352,8500,INR,1,NaN,NaN,NaN,Posted 12 days ago,"Zero Brokerage.\n\n2 Room set, Govt bijali Met...",0.0,Semi-Furnished
3,3 BHK Independent House,1020,Model Town,Delhi,28.712898,77.180000,48000,INR,3,NaN,NaN,NaN,Posted a year ago,Itâs a 3 bhk independent house situated in M...,0.0,Furnished
4,2 BHK Apartment,810,Sector 13 Rohini,Delhi,28.723539,77.131424,20000,INR,2,NaN,NaN,NaN,Posted a year ago,Well designed 2 bhk multistorey apartment is a...,0.0,Unfurnished


In [8]:
# Define features and target
X = df[['house_type', 'house_size', 'location', 'SecurityDeposit', 'Status', 'numBathrooms', 'city']]
y = df['price']

In [9]:
# Preprocessing
categorical_features = ['house_type', 'location', 'Status', 'city']
numerical_features = ['house_size', 'SecurityDeposit', 'numBathrooms']

preprocessor = ColumnTransformer([
    ('num', StandardScaler(), numerical_features),
    ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
])

X_processed = preprocessor.fit_transform(X)

In [10]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
# Define numerical and categorical columns
numerical_cols = ['house_size', 'SecurityDeposit', 'numBathrooms']
categorical_cols = ['house_type', 'location', 'Status', 'city']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),  
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)  
    ]
)

# Apply preprocessing to the training and test datasets
X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)

In [12]:
# Model Building
from keras import models
from keras import layers

In [13]:
ANN_reg = models.Sequential()
ANN_reg.add(layers.Dense(units=128,activation='relu',input_dim=X_train_processed.shape[1]))
ANN_reg.add(layers.Dense(units=64,activation='relu'))
ANN_reg.add(layers.Dense(units=1,activation='linear'))

C:\Users\nadip\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [14]:
ANN_reg.compile(optimizer='sgd',loss='MAE',metrics=[keras.metrics.RootMeanSquaredError])

In [15]:
ANN_reg.fit(X_train_processed, y_train, batch_size=32, epochs=10)

Epoch 1/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 217870.7344 - root_mean_squared_error: 347838.8438
Epoch 2/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 155588.4375 - root_mean_squared_error: 261164.4375
Epoch 3/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 138108.7656 - root_mean_squared_error: 230248.4531
Epoch 4/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 171440.7812 - root_mean_squared_error: 292279.0625
Epoch 5/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 145503.3281 - root_mean_squared_error: 245676.9375
Epoch 6/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 130386.0703 - root_mean_squared_error: 227933.0156
Epoch 7/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 115355.3750 - root_mean_squared_error: 202356.1094
Epoch 8/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 135137.6250 - root_mean_squared_error: 236275.4375
Epoch 9/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 128388.8984 - root_mean_squared_error: 223171.2969
E

In [16]:
ANN_reg.evaluate(X_test_processed,y_test)

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 149212.4844 - root_mean_squared_error: 205024.4375 


[149558.921875, 206224.015625]